In [1]:
import numpy as np
import tonic
import tonic.transforms as transforms
import pandas as pd
from torch.utils.data import Dataset, DataLoader
#from tonic.dataset import Dataset
from typing import Callable, Optional
import torch
import matplotlib.pyplot as plt
from torchvision.ops import masks_to_boxes
from torchvision.utils import draw_bounding_boxes
import matplotlib.patches as patches

In [2]:
#main_dir = "/media/user/EVIMO/packaged/npz/left_camera/imo/eval" #"/media/user/EVIMO/raw/imo/eval/"
#scene_dir = "scene13_dyn_test_00_" # scene15_dyn_test01
#sub_dir = "left_camera/ground_truth_000000"
#dir = main_dir + scene_dir + sub_dir
dir = "/media/user/EVIMO/packaged/npz/left_camera/imo/eval/scene13_dyn_test_05_000000"

save_to_dir = "./data/EVIMO/left_cam/scene13_test5"
sensor_size = [640, 480, 2]
batch_size = 4
framerate = 200
device = 'cpu'

In [3]:
xy = np.load(dir + "/dataset_events_xy.npy")
polarity = np.load(dir + "/dataset_events_p.npy")
time = np.load(dir + "/dataset_events_t.npy")

In [4]:
print(xy.shape, xy.dtype)
print(time.shape, time.dtype)
print(polarity.shape, polarity.dtype)

(11925919, 2) uint16
(11925919,) float32
(11925919,) uint8


In [5]:
df = pd.DataFrame(columns=["x", "y", "t", "p"])
df["x"] = xy[:, 0]
df["y"] = xy[:, 1]
df["t"] = time
df["p"] = polarity

In [6]:
# Convert timestamp from seconds into microseconds
#df["t"] = df["t"].apply(lambda t: t * 1000000)

In [7]:
df

x    y         t  p
0         106  385  2.250000  1
1         147  385  2.250000  1
2         124  384  2.250000  1
3         217  384  2.250000  0
4         323  386  2.250000  1
...       ...  ...       ... ..
11925914  631  218  4.689997  1
11925915  631  218  4.689997  1
11925916   33  398  4.689997  1
11925917  194  266  4.689998  0
11925918    2  370  4.689999  1

[11925919 rows x 4 columns]

In [8]:
dataset_info = dict(np.load(dir + "/dataset_info.npz", allow_pickle=True))

In [9]:
frames = dataset_info["meta"].tolist()["frames"]

In [10]:
dataset_masks = dict(np.load(dir + "/dataset_mask.npz", allow_pickle=True))

What this is: masks are not always available. Sometimes they just don't exist. Also, the time associated for each mask differs. Therefore, I will need to line up masks with timestamps and bin the data differently and by hand

In [11]:
num_bins_per_frame = 4
dataset = []

for i in range(len(frames)-1):
    frame = frames[i]
    start_timestamp = frame["ts"]
    end_timestamp = frames[i+1]["ts"]
    mask_name = "mask_0000000{:03d}".format(i)
    if mask_name in dataset_masks:
        #print(start_timestamp, end_timestamp)
        mask = dataset_masks[mask_name]/1000
        objs_in_mask = np.unique(mask)
        
        events = df[(df.t >= start_timestamp) & (df.t < end_timestamp)].to_records(index=False)

        transform = transforms.ToFrame(sensor_size=sensor_size, n_event_bins=num_bins_per_frame)
        events = transform(events)
        
        data = {"id": i, "start_timestamp": start_timestamp, "end_timestamp": end_timestamp,
               "objs_in_mask": objs_in_mask, "mask": mask, "binned_events": events}
        dataset.append(data)
        

In [12]:
for i, data in enumerate(dataset):
    np.save(save_to_dir + "/" + str(i), data, allow_pickle=True)

In [13]:
np.save(save_to_dir + "/length", len(dataset))

In [14]:
#targets = np.asarray([[0, 1] if 23 in m else [1, 0] for m in masks])

In [15]:
#targets.shape

In [16]:
#for i, binned_event in enumerate(binned_events):
#    np.save(save_to_dir + "/" + str(i), binned_event)

In [17]:
#np.save(save_to_dir + "/labels", targets)